In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "/home/dhamarkin/Yo/Paper_Pablo/complexity/data_clean/"

In [3]:
# Read the data
# Rename some columns
# Only work with the useful variables
df_EC = pd.read_csv(path + "PA_ruc.txt", delimiter='\t', low_memory=False) # PA RUC
df_EC = df_EC.rename(columns={"codigo_ciiu4n4_clase": "ciiu_4n4", "idanonimo": "id"})
df_EC = df_EC[["id", "ciiu_4n4", "codigo_canton"]]

In [4]:
df_EC

,id,ciiu_4n4,codigo_canton
0,1,L6820,21701
1,2,L6820,20101
2,3,A0141,20101
3,4,L6810,20101
4,5,G4610,21701
...,...,...,...
4270730,4268683,S9609,10901
4270731,4268684,V0300,21701
4270732,4268685,V0300,10804
4270733,4268686,V0300,21701


## Diference 1

In [5]:
matrix_d1 = {}
years = list(range(2007, 2015+1))
for idx, year in enumerate(years):
    if idx < len(years) - 1:
        df_n = pd.read_csv(path+f"totaltax_noND({year}).csv", low_memory=False)
        df_n1 = pd.read_csv(path+f"totaltax_noND({year+1}).csv", low_memory=False)
        
        # id_inf n
        id_inf_n = list(df_n["id_inf"].unique())  # <- informante 
        # id_inf n+1
        id_inf_n1 = list(df_n1["id_inf"].unique())

        # id_pov n
        id_prov_n = list(df_n["id_prov"].unique()) # <- proveedor
        # id_prov n+1
        id_prov_n1 = list(df_n1["id_prov"].unique())

        # id n
        id_n = id_inf_n + id_prov_n  # <- informante + proveedor unicos
        id_n = set(id_n)
        num_list_ids_n = len(id_n)

        # id n+1
        id_n1 = id_inf_n1 + id_prov_n1
        id_n1 = set(id_n1)
        num_list_ids_n1 = len(id_n1)

        # id df n
        data_n = df_EC[df_EC['id'].isin(id_n)].reset_index(drop=True)
        num_df_ids_n = data_n.shape[0]

        # id df n+1
        data_n1 = df_EC[df_EC['id'].isin(id_n1)].reset_index(drop=True)  # Check coments in 03_SRI
        num_df_ids_n1 = data_n1.shape[0]

        # columns n
        # eliminar datos con ciiu no reconocido
        columns_n = data_n[["ciiu_4n4", "codigo_canton"]]
        columns_n = columns_n[columns_n["ciiu_4n4"] != "99999"]

        # columns n+1
        # eliminar datos con ciiu no reconocido
        columns_n1 = data_n1[["ciiu_4n4", "codigo_canton"]]
        columns_n1 = columns_n1[columns_n1["ciiu_4n4"] != "99999"]
        
        # generar matriz con 0 cuando no existe ese ciuu en el cantón y 1 cuando sí existe
        matrix_pivot_n = columns_n.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)
        matrix_pivot_n1 = columns_n1.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)

        # calcular la cantidad de filas y columnas de la matriz
        rows_n, columns_n = matrix_pivot_n.shape
        rows_n1, columns_n1 = matrix_pivot_n1.shape

        # generar una matriz más grande que contenga los valores de ciiu y codigo canton de ambas matrices
        index_set = set(matrix_pivot_n.index).union(set(matrix_pivot_n1.index))
        column_set = set(matrix_pivot_n.columns).union(set(matrix_pivot_n1.columns))
        matriz_pivot_n_n1 = pd.DataFrame(0, index=list(index_set), columns=list(column_set))

        # si en el año n hay 0 y en el año n+1 hay 1 se pone 1 en esa posición, se crea la empresa pionera
        for index in matriz_pivot_n_n1.index:
            for column in matriz_pivot_n_n1.columns:
                if index in matrix_pivot_n.index and column in matrix_pivot_n.columns and \
                index in matrix_pivot_n1.index and column in matrix_pivot_n1.columns:
                    if matrix_pivot_n.loc[index, column] == 0 and matrix_pivot_n1.loc[index, column] == 1:
                        matriz_pivot_n_n1.loc[index, column] = 1
        
        count_0 = (matriz_pivot_n_n1 == 0).sum().sum()  
        count_1 = (matriz_pivot_n_n1 == 1).sum().sum() 
        percent_1 = (count_1 / (count_0 + count_1)) * 100
        print(f"{year}-{year+1} Percent 1 in relation with 0:", percent_1, "%")
        print("\n")

        matrix_d1[f"{year}-{year+1}"] = {
            "matrix": matriz_pivot_n_n1,
            "relation": f"{count_0}/({count_0}+{count_1}={percent_1}"
        }

2007-2008 Percent 1 in relation with 0: 3.2883724446268316 %


2008-2009 Percent 1 in relation with 0: 2.77526395173454 %


2009-2010 Percent 1 in relation with 0: 2.8159628550853926 %


2010-2011 Percent 1 in relation with 0: 2.573059188959706 %


2011-2012 Percent 1 in relation with 0: 2.64722004277684 %


2012-2013 Percent 1 in relation with 0: 2.526395173453997 %


2013-2014 Percent 1 in relation with 0: 2.696703909416483 %


2014-2015 Percent 1 in relation with 0: 2.890365093118535 %




In [10]:
matrix_d1['2007-2008']["matrix"]

,20401,11320,20601,31604,21805,32105,10712,10709,20502,20610,...,22302,10907,21709,20304,20507,21701,31503,11206,11302,10910
C1071,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
C1074,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q8810,0,0,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
R9101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C2816,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N8030,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0910,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
K6411,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
G4751,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


## Difference 2

In [ ]:
matrix_d2 = {}
years = list(range(2007, 2015))
for idx, year in enumerate(years):
    if idx < len(years) - 2:
        df_n = pd.read_csv(path+f"totaltax_noND({year}).csv", low_memory=False)
        df_n2 = pd.read_csv(path+f"totaltax_noND({year+2}).csv", low_memory=False)
        
        # id_inf n
        id_inf_n = list(df_n["id_inf"].unique())
        # id_inf n+2
        id_inf_n2 = list(df_n2["id_inf"].unique())

        # id_pov n
        id_prov_n = list(df_n["id_prov"].unique())
        # id_prov n+2
        id_prov_n2 = list(df_n2["id_prov"].unique())

        # id n
        id_n = id_inf_n + id_prov_n
        id_n = set(id_n)
        num_list_ids_n = len(id_n)

        # id n+2
        id_n2 = id_inf_n2 + id_prov_n2
        id_n2 = set(id_n2)
        num_list_ids_n2 = len(id_n2)

        # id df n
        data_n = df_EC[df_EC['id'].isin(id_n)].reset_index(drop=True)
        num_df_ids_n = data_n.shape[0]

        # id df n+2
        data_n2 = df_EC[df_EC['id'].isin(id_n2)].reset_index(drop=True)
        num_df_ids_n2 = data_n2.shape[0]

        # columns n
        columns_n = data_n[["ciiu_4n4", "codigo_canton"]]
        columns_n = columns_n[columns_n["ciiu_4n4"] != "99999"]

        # columns n+2
        columns_n2 = data_n2[["ciiu_4n4", "codigo_canton"]]
        columns_n2 = columns_n2[columns_n2["ciiu_4n4"] != "99999"]

        matrix_pivot_n = columns_n.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)
        matrix_pivot_n2 = columns_n2.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)

        rows_n, columns_n = matrix_pivot_n.shape
        rows_n2, columns_n2 = matrix_pivot_n2.shape

        index_set = set(matrix_pivot_n.index).union(set(matrix_pivot_n2.index))
        column_set = set(matrix_pivot_n.columns).union(set(matrix_pivot_n2.columns))
        matriz_pivot_n_n2 = pd.DataFrame(0, index=list(index_set), columns=list(column_set))

        for index in matriz_pivot_n_n2.index:
            for column in matriz_pivot_n_n2.columns:
                if index in matrix_pivot_n.index and column in matrix_pivot_n.columns and \
                index in matrix_pivot_n2.index and column in matrix_pivot_n2.columns:
                    if matrix_pivot_n.loc[index, column] == 0 and matrix_pivot_n2.loc[index, column] == 1:
                        matriz_pivot_n_n2.loc[index, column] = 1
        
        count_0 = (matriz_pivot_n_n2 == 0).sum().sum()  
        count_1 = (matriz_pivot_n_n2 == 1).sum().sum()  
        percent_1 = (count_1 / (count_0 + count_1)) * 100
        print("Percent 1 in relation with 0:", percent_1, "%")
        print("\n")

        matrix_d2[f"{year}-{year+2}"] = {
            "matrix": matriz_pivot_n_n2,
            "relation": f"{count_0}/({count_0}+{count_1}={percent_1}"
        }

Percent 1 in relation with 0: 4.755967799142313 %


Percent 1 in relation with 0: 4.2486645672338 %


Percent 1 in relation with 0: 4.024994613229907 %


Percent 1 in relation with 0: 3.8536595826810487 %


Percent 1 in relation with 0: 3.727389000548145 %


Percent 1 in relation with 0: 3.7732409770324464 %




## Difference 3

In [8]:
matrix_d3 = {}
years = list(range(2007, 2015))
for idx, year in enumerate(years):
    if idx < len(years) - 3:
        df_n = pd.read_csv(path+f"totaltax_noND({year}).csv", low_memory=False)
        df_n3 = pd.read_csv(path+f"totaltax_noND({year+3}).csv", low_memory=False)
        
        # id_inf n
        id_inf_n = list(df_n["id_inf"].unique())
        # id_inf n+3
        id_inf_n3 = list(df_n3["id_inf"].unique())

        # id_pov n
        id_prov_n = list(df_n["id_prov"].unique())
        # id_prov n+3
        id_prov_n3 = list(df_n3["id_prov"].unique())

        # id n
        id_n = id_inf_n + id_prov_n
        id_n = set(id_n)
        num_list_ids_n = len(id_n)

        # id n+3
        id_n3 = id_inf_n3 + id_prov_n3
        id_n3 = set(id_n3)
        num_list_ids_n3 = len(id_n3)

        # id df n
        data_n = df_EC[df_EC['id'].isin(id_n)].reset_index(drop=True)
        num_df_ids_n = data_n.shape[0]

        # id df n+3
        data_n3 = df_EC[df_EC['id'].isin(id_n3)].reset_index(drop=True)
        num_df_ids_n3 = data_n3.shape[0]

        # columns n
        columns_n = data_n[["ciiu_4n4", "codigo_canton"]]
        columns_n = columns_n[columns_n["ciiu_4n4"] != "99999"]

        # columns n+3
        columns_n3 = data_n3[["ciiu_4n4", "codigo_canton"]]
        columns_n3 = columns_n3[columns_n3["ciiu_4n4"] != "99999"]

        matrix_pivot_n = columns_n.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)
        matrix_pivot_n3 = columns_n3.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)

        rows_n, columns_n = matrix_pivot_n.shape
        rows_n2, columns_n3 = matrix_pivot_n3.shape

        index_set = set(matrix_pivot_n.index).union(set(matrix_pivot_n3.index))
        column_set = set(matrix_pivot_n.columns).union(set(matrix_pivot_n3.columns))
        matriz_pivot_n_n3 = pd.DataFrame(0, index=list(index_set), columns=list(column_set))

        for index in matriz_pivot_n_n3.index:
            for column in matriz_pivot_n_n3.columns:
                if index in matrix_pivot_n.index and column in matrix_pivot_n.columns and \
                index in matrix_pivot_n3.index and column in matrix_pivot_n3.columns:
                    if matrix_pivot_n.loc[index, column] == 0 and matrix_pivot_n3.loc[index, column] == 1:
                        matriz_pivot_n_n3.loc[index, column] = 1
        
        count_0 = (matriz_pivot_n_n3 == 0).sum().sum()  
        count_1 = (matriz_pivot_n_n3 == 1).sum().sum()  
        percent_1 = (count_1 / (count_0 + count_1)) * 100
        print("Percent 1 in relation with 0:", percent_1, "%")
        print("\n")

        matrix_d2[f"{year}-{year+3}"] = {
            "matrix": matriz_pivot_n_n3,
            "relation": f"{count_0}/({count_0}+{count_1}={percent_1}"
        }

KeyboardInterrupt: 

## Difference 4

In [ ]:
matrix_d4 = {}
years = list(range(2007, 2015))
for idx, year in enumerate(years):
    if idx < len(years) - 4:
        df_n = pd.read_csv(path+f"totaltax_noND({year}).csv", low_memory=False)
        df_n4 = pd.read_csv(path+f"totaltax_noND({year+4}).csv", low_memory=False)
        
        # id_inf n
        id_inf_n = list(df_n["id_inf"].unique())
        # id_inf n+4
        id_inf_n4 = list(df_n4["id_inf"].unique())

        # id_pov n
        id_prov_n = list(df_n["id_prov"].unique())
        # id_prov n+4
        id_prov_n4 = list(df_n4["id_prov"].unique())

        # id n
        id_n = id_inf_n + id_prov_n
        id_n = set(id_n)
        num_list_ids_n = len(id_n)

        # id n+4
        id_n4 = id_inf_n4 + id_prov_n4
        id_n4 = set(id_n4)
        num_list_ids_n4 = len(id_n4)

        # id df n
        data_n = df_EC[df_EC['id'].isin(id_n)].reset_index(drop=True)
        num_df_ids_n = data_n.shape[0]

        # id df n+4
        data_n4 = df_EC[df_EC['id'].isin(id_n4)].reset_index(drop=True)
        num_df_ids_n4 = data_n4.shape[0]

        # columns n
        columns_n = data_n[["ciiu_4n4", "codigo_canton"]]
        columns_n = columns_n[columns_n["ciiu_4n4"] != "99999"]

        # columns n+4
        columns_n4 = data_n4[["ciiu_4n4", "codigo_canton"]]
        columns_n4 = columns_n4[columns_n4["ciiu_4n4"] != "99999"]

        matrix_pivot_n = columns_n.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)
        matrix_pivot_n4 = columns_n4.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)

        index_set = set(matrix_pivot_n.index).union(set(matrix_pivot_n4.index))
        column_set = set(matrix_pivot_n.columns).union(set(matrix_pivot_n4.columns))
        matriz_pivot_n_n4 = pd.DataFrame(0, index=list(index_set), columns=list(column_set))

        for index in matriz_pivot_n_n4.index:
            for column in matriz_pivot_n_n4.columns:
                if index in matrix_pivot_n.index and column in matrix_pivot_n.columns and \
                index in matrix_pivot_n4.index and column in matrix_pivot_n4.columns:
                    if matrix_pivot_n.loc[index, column] == 0 and matrix_pivot_n4.loc[index, column] == 1:
                        matriz_pivot_n_n4.loc[index, column] = 1
        
        count_0 = (matriz_pivot_n_n4 == 0).sum().sum()  
        count_1 = (matriz_pivot_n_n4 == 1).sum().sum()   
        percent_1 = (count_1 / (count_0 + count_1)) * 100
        print("Percent 1 in relation with 0:", percent_1, "%")
        print("\n")

        matrix_d2[f"{year}-{year+4}"] = {
            "matrix": matriz_pivot_n_n4,
            "relation": f"{count_0}/({count_0}+{count_1}={percent_1}"
        }

## Difference 5

In [ ]:
matrix_d5 = {}
years = list(range(2007, 2015))
for idx, year in enumerate(years):
    if idx < len(years) - 5:
        df_n = pd.read_csv(path+f"totaltax_noND({year}).csv", low_memory=False)
        df_n5 = pd.read_csv(path+f"totaltax_noND({year+5}).csv", low_memory=False)
        
        # id_inf n
        id_inf_n = list(df_n["id_inf"].unique())
        # id_inf n+5
        id_inf_n5 = list(df_n5["id_inf"].unique())

        # id_pov n
        id_prov_n = list(df_n["id_prov"].unique())
        # id_prov n+5
        id_prov_n5 = list(df_n5["id_prov"].unique())

        # id n
        id_n = id_inf_n + id_prov_n
        id_n = set(id_n)
        num_list_ids_n = len(id_n)

        # id n+5
        id_n5 = id_inf_n5 + id_prov_n5
        id_n5 = set(id_n5)
        num_list_ids_n5 = len(id_n5)

        # id df n
        data_n = df_EC[df_EC['id'].isin(id_n)].reset_index(drop=True)
        num_df_ids_n = data_n.shape[0]

        # id df n+5
        data_n5 = df_EC[df_EC['id'].isin(id_n5)].reset_index(drop=True)
        num_df_ids_n5 = data_n5.shape[0]

        # columns n
        columns_n = data_n[["ciiu_4n4", "codigo_canton"]]
        columns_n = columns_n[columns_n["ciiu_4n4"] != "99999"]

        # columns n+5
        columns_n5 = data_n5[["ciiu_4n4", "codigo_canton"]]
        columns_n5 = columns_n5[columns_n5["ciiu_4n4"] != "99999"]

        matrix_pivot_n = columns_n.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)
        matrix_pivot_n5 = columns_n5.pivot_table(index='ciiu_4n4', columns='codigo_canton', aggfunc=lambda x: 1, fill_value=0)

        index_set = set(matrix_pivot_n.index).union(set(matrix_pivot_n5.index))
        column_set = set(matrix_pivot_n.columns).union(set(matrix_pivot_n5.columns))
        matriz_pivot_n_n5 = pd.DataFrame(0, index=list(index_set), columns=list(column_set))

        for index in matriz_pivot_n_n5.index:
            for column in matriz_pivot_n_n5.columns:
                if index in matrix_pivot_n.index and column in matrix_pivot_n.columns and \
                index in matrix_pivot_n5.index and column in matrix_pivot_n5.columns:
                    if matrix_pivot_n.loc[index, column] == 0 and matrix_pivot_n5.loc[index, column] == 1:
                        matriz_pivot_n_n5.loc[index, column] = 1
        
        count_0 = (matriz_pivot_n_n5 == 0).sum().sum()  
        count_1 = (matriz_pivot_n_n5 == 1).sum().sum() 
        percent_1 = (count_1 / (count_0 + count_1)) * 100
        print("Percent 1 in relation with 0:", percent_1, "%")
        print("\n")

        matrix_d2[f"{year}-{year+5}"] = {
            "matrix": matriz_pivot_n_n5,
            "relation": f"{count_0}/({count_0}+{count_1}={percent_1}"
        }